In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from qbayes_tools import generate_cond_keys

ModuleNotFoundError: No module named 'qbayes_tools'

In [37]:
data = pd.read_csv('data/lesser_model_data.csv')

In [38]:
"""
IMPORTANT NOTE Y'ALL::maybe once we have all three models we can move the get_lesser_model_states
to some sort of get states code and just make this a place to get general probabilities 
(and call get_states and probabilities functions from a network specific circuit builder directly,
OR a general circuit builder would be the dream!!)
"""

"\nIMPORTANT NOTE Y'ALL::maybe once we have all three models we can move the get_lesser_model_states\nto some sort of get states code and just make this a place to get general probabilities \n(and call get_states and probabilities functions from a network specific circuit builder directly,\nOR a general circuit builder would be the dream!!)\n"

In [40]:
def get_lesser_model_states():
    statedataStayHome = {'MarHome' : data['MarHome'], 'AprHome' : data['AprHome'], 'MayHome' : data['MayHome'], 'JunHome' : data['JunHome']}
    statesStayHome = pd.DataFrame(data=statedataStayHome)

    statedataTests = {'MarTest' : data['MarTest'], 'AprTest' : data['AprTest'], 'MayTest' : data['MayTest'], 'JunTest' : data['JunTest']}
    statesTests = pd.DataFrame(data=statedataTests)

    statedataCases = {'MarCases' : data['MarCases'], 'AprCases' : data['AprCases'], 'MayCases' : data['MayCases'], 'JunCases' : data['JunCases']}
    statesCases = pd.DataFrame(data=statedataCases) # 0 = increasing. 1 = flat or decreasing
    
    return np.ndarray.flatten(np.array(statesCases)), np.ndarray.flatten(np.array(statesTests)), np.ndarray.flatten(np.array(statesStayHome))

In [41]:
def get_probabilities(state_data):
    ############################################
    # USE THIS FUNCTION TO FIND THE PROBABILITIES FOR AN INDIVIDUAL NODE IN THE NETWORK
    
    ### INPUT ###
    # state_data: 1d array indicating the state corresponding to individual data points
    #             (note: for an n-state system the array can contain the integers 
    #             0, 1, ..., n-1, but must contain n-1 at least once) [note to contributors: 
    #             maybe add an optional input "n" to specify a maximum state that is never attained?]

    ### OUTPUT ###
    # probs: array of length n with probabilities for states 0 through n-1 in increasing order
    ############################################

    num_total = len(state_data)  #total number of data points with which to calculate probabilities
    s = max(state_data) + 1  #number of different states will be the maximum integer included in the state data plus one

    probs = np.zeros(s)
    for i in range(s): #loop through different allowed states
        probs[i] = np.shape(np.where(state_data == i))[1]/num_total

    assert round(np.sum(probs), 3) == 1. 
    
    return probs

In [46]:
def get_conditional_probability(child, *ps):
    ############################################
    ### THIS FUNCTION CALCULATES CONDITIONAL PROBABILITIES FOR CHILD NODE
    ### THAT HAS s_m STATES AND m PARENT NODES EACH WITH s_i STATES WHERE i = 0, ..., m-1
    
    ### INPUTS ###
    # child    array    1d array with n entries specifying the state of the child node at each data point
    # *ps      array(s) variable number of 1d arrays with n entries each, specifying the states of parent nodes at each data point
    
    ### OUTPUT ###
    # a dictionary of conditional probabilities
    ############################################

    #we might want to add some assert statements checking that all inputs have the same shape
    
    m = len(ps) #number of parent nodes
    s_0 = max(child) + 1 #number of child nodes
    
    s_i = np.zeros(m)
    for i in range(m):
        s_i[i] = max(ps[i]) + 1  #number of states for this node will be the maximum state noted in that data plus one
    keys = tools.generate_cond_keys(s_0,s_i)
    
    cond_probs = {} #initialize a dictionary for conditional probabilities
    for key in keys:
        numer, tot = 0, 0
        n = len(c)
        for i in range(n):
            all_ps = True
            for j in range(len(ps)):
                p = ps[j]
                if p[i] != int(key.split("|")[1].split(",")[j]):
                    all_ps = False
                    break
            if all_ps:
                tot += 1
                if child[i] == int(key.split("|")[0]):
                    numer += 1
            
        cond_probs.update({key : num/tot})

    """
    # Make graph for circuit input
    ProbA1, ProbA0 = get_probabilities(Astates)
    ProbB1, ProbB0 = get_probabilities(Bstates)
    
    graph = { 'StayAtHome': ([], [ProbA0, ProbA1]), # P(A = 0), P(A = 1)
                'Testing': ([], [ProbB0, ProbB1]),  #P(B = 0), P(B = 1)
                #P(C=0|A=0,B=0), P(C=1|A=0,B=0), P(C=0|A=0,B=1), P(C=1|A=0,B=1), P(C=0|A=1,B=0), P(C=1|A=1,B=0), P(C=0|A=1,B=1), P(C=1|A=1,B=1)
                'Cases': (['StayAtHome','Testing'], [P000, P100, P010, P110, P001, P101, P011, P111])
              }
    
    return states, conditional_probabilities, graph
    """
    
    return cond_probs

In [47]:
A,B,C = get_lesser_model_states()

print(get_probabilities(C))
print(get_conditional_probability(C, B, A))

[0.645 0.355]


NameError: name 'tools' is not defined

In [ ]:
""" # alternate

def f(c, *ps):
	for key in keys:
		num_c, tot_c = 0, 0
		n = len(c)
		for i in range(n):
			all_ps = all([ps[j][i] == key[j] for j in range(len(ps))])
			if all_ps:
				tot_c += 1
				if c[i] == thing before |:
					num_c += 1

"""

In [6]:
# example: results from running simple model on simulator: 
# {'000': 2783, '001': 1240, '100': 603, '111': 815, '110': 294, '010': 1712, '101': 485, '011': 260}

def get_marginal_0probabilities(state_counts):
    #state_counts: dict, counts for each state from network result (should have 2^n entries)
    #marg_probs: array of length n, marginal probabilities that each qubit is 0,
        #from most significant to least significant qubit
    
    n = len(list(state_counts.keys())[0]) #number of qubits
    prob = np.zeros(n)
    total = sum(state_counts.values())

    for i in range(n):
        for key in state_counts:
            if int(key[i]) == 0:
                prob[i] += state_counts[key]
        prob[i] = prob[i]/total
    
    return prob




These calculations agree with marginal_probabilities function, yay !

PC0_calc = (2783+1240+1712+260)/(2783+1240+603+815+294+1712+485+260)

print(PC0_calc)

PB0_calc = (2783+1240+485+603)/(2783+1240+603+815+294+1712+485+260)

print(PB0_calc)

PA0_calc = (2783+603+294+1712)/(2783+1240+603+815+294+1712+485+260)

print(PA0_calc)


In [1]:
# get marginal probabilities for simulation results - lesser model

results = {'000': 2783, '001': 1240, '100': 603, '111': 815, '110': 294, '010': 1712, '101': 485, '011': 260}
#margProb = marginal_probabilities(results)
#margCompare = marginal_probabilities_general(results)
#print(margProb)
#print(margCompare)

In [6]:
def func(**counts):
    return counts["c001"]

In [7]:
func(c000=4928, c001=82743, c100=48937, c111=9288842)

82743